# 21.5. Встроенные фикстуры pytest
## @pytest.mark фикстура
- **pytest.mark.skip** - помечает тест как пропущенный, то есть при запуске тест не будет выполняться.

In [ ]:
@pytest.mark.skip(reason="Баг в продукте - <ссылка>")
def test_one(): # Это наш тест, который находит тот самый баг
    pass

- **pytest.mark.skipif** - о же самое, что и предыдущая, но мы имеем возможность управлять игнорируемыми тестами

In [ ]:
@pytest.mark.skipif(sys.version_info < (3, 6), reason="Тест требует python версии 3.6 или выше")
def test_python36_and_greater():
    pass

мы можем создать переменную, в которой будет выполняться проверка версии языка, оборачивать в фикстуру pytest и далее использовать её в тестах в виде декоратора:

In [ ]:
minversion = pytest.mark.skipif(
    sys.version_info < (3, 6), reason="at least mymodule-1.1 required"
)

@minversion
def test_python36_and_greater():
    pass

Перечисленные фикстуры могут применяться аналогичным образом к целым классам с тестами. Например, это удобно применять, когда у нас имеется несколько файлов с тестами для различных операционных систем, но мы не знаем заранее, на компьютере с какой ОС будет запускаться пачка тестов.

In [ ]:
@pytest.mark.skipif(env != 'test', reason="Эта фикстура запускается только на тестовом окружении")

- **pytest.mark.xfail** - Помечает тест как падающий.

In [ ]:
@pytest.mark.xfail
def test_flaky():

# с условием
@pytest.mark.xfail(sys.platform == "win32", reason="Ошибка в системной библиотеке") # На платформе Windows ожидаем, что тест будет падать
def test_not_for_windows():

# конкретное исключение
@pytest.mark.xfail(raises=RuntimeError)
def test_x_status_runtime_only():

## Пользовательские группы
необходимо написать, что мы помечаем тест, и дать имя группы (@pytest.mark.auth). Далее необходимо в проекте создать файл pytest.ini, туда внести информацию об описанных в тестах группах. Давайте разберём непосредственно в коде. У нас есть четыре теста, два из них на аутентификацию пользователя, остальные два — это тесты мероприятий. В каждой такой группе соответственно API и UI тесты:

In [ ]:
@pytest.mark.api
@pytest.mark.auth
def test_auth_api():
   pass

@pytest.mark.ui
@pytest.mark.auth
def test_auth_ui():
   pass

@pytest.mark.api
@pytest.mark.event
def test_event_api():
   pass

@pytest.mark.ui
@pytest.mark.event
def test_event_ui():
   pass

В корне проекта создадим файл pytest.ini и добавим туда описание наших категорий. Тесты будут запускаться и без этого файла, но его наличие избавит нас от постоянных предупреждений в отчетах:
```bash
[pytest]
markers =
   api: тесты API
   ui: тесты UI
   event: тесты мероприятий
   auth: тесты авторизации
 ```
 Все, что нам осталось сделать — это научиться фильтровать такие тесты. Например, если нам нужно запустить только API тесты, то в консоли надо набрать:
```bash
pytest test.py -v -m "api" # test.py замените на имя своего файла в проекте
```
Например, если мы хотим запустить только UI тесты авторизации, то команда в консоли будет выглядеть так:
```bash
pytest test.py -v -m "ui and auth"
```
А если нам нужно запустить все виды тесты на модули авторизации и мероприятий, то команда для запуска будет следующая:
```bash
pytest test.py -v -m "auth or event"
```
[Документация](https://pytest-docs-ru.readthedocs.io/ru/latest/example/markers.html)